In [2]:
from sklearn.metrics import f1_score, precision_score, recall_score
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
import random
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from torch.nn.parallel import DataParallel
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
from google.colab import drive
drive.mount('/content/drive')

emotions_data = pd.read_csv('/content/drive/My Drive/Emotion/label70prob.csv')
emotions_data.head(3)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


0        id say maybe made feel foolish would reeeeeeal...
1        joined lds church admit feeling somewhat asham...
2        must admit didnt feel like hugging angry disgu...
3        hate still feel nerve damaged badly enough oft...
4                          im actually feeling little smug
                               ...                        
83357    need know pain feel vain better brighter day f...
83358                     feel type work really suits well
83359    finished taking exams need relax said suddenly...
83360    swear fireworks outside cool breeze siren song...
83361    sometimes feel boring person people love gossi...
Name: text, Length: 83362, dtype: object

In [16]:


import pandas as pd
import re
from sklearn.model_selection import train_test_split

def clean_text(text):
    """
    A function to clean text data, including removing URLs, HTML tags, special characters, and handling non-string inputs.
    """
    # Check if the text is a string
    if not isinstance(text, str):
        return ""  # Return empty string if not

    # Remove URLs
    text = re.sub(r'http\S+|www.\S+', '', text)
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text).strip()
    # Optionally convert text to lowercase
    text = text.lower()
    return text
# Fill NaN values with an empty string before applying the cleaning function
emotions_data['text'] = emotions_data['text'].fillna('')

# Now apply the cleaning function
emotions_data['text'] = emotions_data['text'].apply(clean_text)







# Split the dataset
train_df, val_df = train_test_split(emotions_data, test_size=0.2, random_state=42)

# Check the result of the split and cleaning
print(train_df.head())

       Unnamed: 0  label                                               text
24981       24981      6  cant delete lose feeling satisfaction one reso...
5254         5254      4  invite join share anyone feel could benefit ge...
74513       74513     11     cant feel sweet everything like even sometimes
75398       75398      6                     tell lonely feel contented els
58166       58166      0                            feel deprived love food


In [17]:
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader
import torch

# Initialize tokenizer from the BERT model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Define a dataset class for the Emotions data
class EmotionsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = str(self.texts[index])
        label = self.labels[index]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Create datasets
train_dataset = EmotionsDataset(train_df['text'].values, train_df['label'].values, tokenizer)
val_dataset = EmotionsDataset(val_df['text'].values, val_df['label'].values, tokenizer)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [18]:
from transformers import BertForSequenceClassification

# Load BERT with the sequence classification head; specify the number of labels
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=21)  # Adjust num_labels based on the number of emotion categories

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
from transformers import AdamW, get_linear_schedule_with_warmup
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

# Optimization and scheduler setup
epochs = 10
optimizer = AdamW(model.parameters(), lr=5e-5)
total_steps = len(train_loader) * epochs  # Define the number of epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training loop
for epoch in range(epochs):  # Define epochs variable if not defined
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    avg_train_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch+1}, Loss: {avg_train_loss:.2f}')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Loss: 0.42
Epoch 2, Loss: 0.32
Epoch 3, Loss: 0.24
Epoch 4, Loss: 0.18
Epoch 5, Loss: 0.13
Epoch 6, Loss: 0.09
Epoch 7, Loss: 0.06
Epoch 8, Loss: 0.04
Epoch 9, Loss: 0.02
Epoch 10, Loss: 0.01


In [22]:
# Assuming the training loop has been completed and the model is trained

# Validation function (as defined above)
from sklearn.metrics import classification_report
def evaluate_model(model, dataloader, device):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            labels = batch['labels'].cpu().numpy()
            predictions.extend(preds)
            true_labels.extend(labels)
    return predictions, true_labels

# Evaluate the model
predictions, true_labels = evaluate_model(model, val_loader, device)

# Print classification report
print(classification_report(true_labels, predictions, target_names=['Sadness', 'Joy', 'Love', 'Anger', 'Fear', 'Surprise','Sadness and Joy', 'Sadness and Love ',
                                  'Sadness and Anger', 'Sadness and Fear', 'Sadness and Surprise', 'Joy and Love','Joy and Anger','Joy and Fear', 'Joy and Surprise', 'Love and Anger','Love and Fear' ,
                                          'Love and Surprise', 'Anger and Fear', 'Anger and Surprise','Fear and Surprise']))

                      precision    recall  f1-score   support

             Sadness       0.79      0.82      0.80      1697
                 Joy       0.82      0.84      0.83      2538
                Love       0.75      0.77      0.76       443
               Anger       0.76      0.81      0.78      1239
                Fear       0.80      0.79      0.80       852
            Surprise       0.81      0.84      0.82       488
     Sadness and Joy       0.87      0.85      0.86      4596
   Sadness and Love        0.88      0.75      0.81       205
   Sadness and Anger       0.84      0.82      0.83       883
    Sadness and Fear       0.85      0.87      0.86       751
Sadness and Surprise       0.74      0.82      0.78        38
        Joy and Love       0.92      0.91      0.91      1335
       Joy and Anger       0.81      0.74      0.77       735
        Joy and Fear       0.88      0.82      0.85       356
    Joy and Surprise       0.74      0.71      0.72       172
      L